# Classification Power Encoding

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio
#%matplotlib notebook
#%matplotlib inline
#%load_ext autoreload
#%autoreload 2
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *

from brainpipe.statistics import *
from os import path
from mne import *

## Classification across time for all freq bands and subjects
### Analysis subject by subject

In [ ]:
# Importing files 
st = study('Olfacto')
pathfiles = path.join(st.path, 'feature/3_Power_Encoding_Odor_rest_th40_art400_30_250_across_time_700ms_step100ms/')
elecfiles = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_concatOK/')
path2save = path.join(st.path, 'classified/5_Classif_Windows_700ms_Encoding_across_time__LDA_ALLregions/')

subjects = [ 'LEFC','CHAF','SEMC', 'VACJ','PIRJ', 'MICP',]
nfreq = 6

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

classif = 'lda'
for su in subjects:
    for elec_num in range(n_elec[su]):
        for freq in range(nfreq):
            #files & data to load
            odor_data = np.load(path.join(pathfiles, su+'_E1E2_concat_all_bipo_odor_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
            rest_data = np.load(path.join(pathfiles, su+'_E1E2_concat_all_bipo_rest_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
            print ('odor shape: ', odor_data.shape, 'rest shape: ', rest_data.shape)
            elec = np.load(path.join(elecfiles, su+'_E1E2_concat_all_bipo.npz'))['channel'][elec_num]
            elec_label = np.load(path.join(elecfiles, su+'_E1E2_concat_all_bipo.npz'))['label'][elec_num]
            freq_name = np.load(path.join(pathfiles, su+'_E1E2_concat_all_bipo_odor_power.npz'))['fname'][freq]
            print ('elec ', elec, 'elec_label ', elec_label)

# =============================  CLASSIFICATION COMPUTATION ============================================================           
            #Repeat the rest by the nb of temporal time points (for swap function)
            rest_rep = np.tile(rest_data, (odor_data.shape[0],1))
            print('-> Shape of repeated rest :', rest_rep.shape)

            #create a data matrix, concatenate along the trial dimension
            odor_rest = np.concatenate((odor_data, rest_rep), axis=1)
            odor_rest = odor_rest.swapaxes(0,1)
            print ('Size of the concatenated data: ', odor_rest.shape, 'Number of features : ', odor_rest.shape[1])

            #create label vector (0 for rest and 1 for odor)
            label = [1]*rest_rep.shape[1] + [0]*odor_data.shape[1]
            print ('Size of label for classif: ', len(label))

            # Define a cross validation:
            cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)
            # Define classifier technique
            clf = defClf(label, clf=classif,) #,n_tree=200, random_state=100)
            #Classify rest and odor
            cl = classify(label, clf=clf, cvtype=cv)

            # Evaluate the classifier on data:
            da,pvalue,daperm = cl.fit(odor_rest, n_perm=100,method='label_rnd',mf=False)
            print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape, 'daperm', daperm.shape)
            #print (da)
            
            #Save information
            np.save(path2save+su+'_da_Odor_vs_Rest__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')',da)
            np.save(path2save+su+'_pvalue_Odor_vs_Rest__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')', pvalue)
            np.save(path2save+su+'_da_perm_Odor_vs_Rest__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')', daperm)

# =========================== PLOT POWER ANALYSIS & DECODING ACCURACY ===================================================

            #Parameters figure
            xfmt = ScalarFormatter(useMathText=True)
            xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(15,4))
            step = 3000/ odor_data.shape[0]
            time = np.arange(0, 3000, step)
            print (len(time), odor_data.shape, rest_rep.shape)
            title = 'Power and DA for '+str(freq_name)+' PIRJ '+classif+' '+str(elec_label)+' ('+str(elec_num)+')'
            fig.suptitle(title, fontsize=16)

            # Plot the power and da :
            plt.subplot(131)
            plt.plot(time, rest_rep, color='b', marker='*',)
            plt.plot(time, odor_data, color='r', marker='o',)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

            #concatenate odor (red) then rest (blue)
            plt.subplot(132)
            BorderPlot(time, odor_rest, y=label, xlabel='Time (ms)',ylabel = r' $\mu$V',
                       alpha=0.3,linewidth=2,color=['b','r'],kind='sem')
            rmaxis(plt.gca(), ['right', 'top'])
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

            plt.subplot(133)
            BorderPlot(time, da, color='darkslateblue', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
                       ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=[1],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'])
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            plt.plot(time, daperm.max()*np.ones((len(time),1)), '--', color='dimgrey', linewidth=2, label=str(daperm.max())+' p < .01')
            print(daperm.max())
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
            
# =========================== SAVE FIGURES & CLEAN MEMORY ==========================================================================
            
            #Save the plot
            fname = path.join(path2save, su + '_'+freq_name+'_'+str(elec_label)+'_('+str(elec_num)+')_'+'0.01.png')
            fig.savefig(fname, dpi=300, bbox_inches='tight')
            print ('saving --»' ,fname)
            plt.clf()
            plt.close()
            del odor_data, rest_data, elec, elec_label, freq_name, da, daperm, pvalue, rest_rep, odor_rest, label
            


## Classification across time for all freq bands and subjects
### Exploration EARLY and LATE

In [2]:
# Importing files 
st = study('Olfacto')
pathfiles = path.join(st.path, 'feature/6_Power_E1E2_Odor_Rest_Explo_2blocks_700_200/')
elecfiles = path.join(st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_learning2blocks/')
path2save = path.join(st.path, 'classified/8_Classif_Explo_Early_Late_700ms_200ms_across_time__LDA/')

subjects = [ 'LEFC','CHAF','SEMC', 'VACJ','PIRJ', 'MICP',]
nfreq = 6
# n_elec = {
# 'CHAF' : [36,95],
# 'VACJ' : [3,61], 
# 'SEMC' : [0,6,25],
# 'PIRJ' : [96],
# 'LEFC' : [0,6,88,91],
# 'MICP' : [88],
#             }

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

classif = 'lda'
for su in subjects:
    for elec_num in range(n_elec[su]):
        for freq in range(nfreq):
            #files & data to load
            early_data = np.load(path.join(pathfiles, su+'_E1E2_concat_early_bipo_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
            late_data = np.load(path.join(pathfiles, su+'_E1E2_concat_late_bipo_power.npz'))['xpow'][freq,elec_num] #take power for one freq band, one elec
            print ('early shape: ', early_data.shape, 'late shape: ', late_data.shape)
            elec = np.load(path.join(elecfiles, su+'_E1E2_concat_early_bipo.npz'))['channel'][elec_num]
            elec_label = np.load(path.join(elecfiles, su+'_E1E2_concat_early_bipo.npz'))['label'][elec_num]
            freq_name = np.load(path.join(pathfiles, su+'_E1E2_concat_early_bipo_power.npz'))['fname'][freq]
            print ('elec ', elec, 'elec_label ', elec_label)
            
# =======================================  STATISTICS FOR POWER  =====================================
            #reshape data to have the exact same nb od trials (mandatory for t-tests)
            if np.array_equal(early_data, late_data)==False:
                early_data_rand = early_data[:, np.random.randint(early_data.shape[1], size=late_data.shape[1])] #reshape early_data to fit late_data shape
                print ('rand early matrix', early_data_rand.shape)
            X = early_data_rand.swapaxes(0,1) - late_data.swapaxes(0,1)
            T0, p_values, H0 = stats.permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

# =============================  CLASSIFICATION COMPUTATION ============================================================           
            #create a data matrix, concatenate along the trial dimension
            early_late = np.concatenate((early_data, late_data), axis=1)
            early_late = early_late.swapaxes(0,1)
            print ('Size of the concatenated data: ', early_late.shape, 'Number of features : ', early_late.shape[1])

            #create label vector (0 for rest and 1 for odor)
            label = [1]*early_data.shape[1] + [0]*late_data.shape[1]
            print ('Size of label for classif: ', len(label))

            # Define a cross validation:
            cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)
            # Define classifier technique
            clf = defClf(label, clf=classif) #,n_tree=200, random_state=100)
            #Classify rest and odor
            cl = classify(label, clf=clf, cvtype=cv)

            # Evaluate the classifier on data:
            da,pvalue,daperm = cl.fit(early_late, n_perm=100,method='label_rnd',mf=False)
            print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape, 'daperm', daperm.shape)
            #print (da)
            
            #Save information
            np.save(path2save+su+'_da_Early_vs_Late__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')',da)
            np.save(path2save+su+'_pvalue_Early_vs_Late__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')', pvalue)
            np.save(path2save+su+'_da_perm_Early_vs_Late__'+str(freq_name)+'_'+classif+'_'+str(elec_label)+'_('+str(elec_num)+')', daperm)

# =========================== PLOT POWER ANALYSIS & DECODING ACCURACY ===================================================

            #Parameters figure
            xfmt = ScalarFormatter(useMathText=True)
            xfmt.set_powerlimits((0,3))
            fig = plt.figure(1,figsize=(7,6))
            step = 3700/ early_data.shape[0]
            time = np.arange(-700, 3000, step)
            title = 'Power and DA for '+str(freq_name)+' '+su+' '+classif+' '+str(elec_label)+' ('+str(elec_num)+')'
            fig.suptitle(title, fontsize=12)

            #concatenate odor (red) then rest (blue)
            plt.subplot(211)
            BorderPlot(time, early_late, y=label, xlabel='Time (ms)',ylabel = r' $\mu$V', legend=['early', 'late'],
                       alpha=0.3,linewidth=2,color=['b','r'],kind='sem')
            addPval(plt.gca(), p_values, p=0.05, x=time, y=0.1, color='0.5', lw=2)
            addPval(plt.gca(), p_values, p=0.01, x=time, y=0.2, color='0.7', lw=2)
            addPval(plt.gca(), p_values, p=0.001, x=time, y=0.3, color='0.9', lw=2)
            addLines(plt.gca(), vLines=[0],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'])
            rmaxis(plt.gca(), ['right', 'top'])
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

            plt.subplot(212)
            BorderPlot(time, da, color='darkslateblue', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
                       ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
            rmaxis(plt.gca(), ['right', 'top'])
            addLines(plt.gca(), vLines=[0],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'])
            plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
            plt.plot(time, np.around(daperm.max(),2)*np.ones((len(time),1)), '--', color='dimgrey', linewidth=2, label=str(np.around(daperm.max(),2))+' p < .01')
            print(daperm.max())
            plt.legend(loc=0, handletextpad=0.1, frameon=False)
                   
# =========================== SAVE FIGURES & CLEAN MEMORY ==========================================================================
            
            #Save the plot
            fname = path.join(path2save, su + '_'+freq_name+'_'+str(elec_label)+'_('+str(elec_num)+')_'+'0.01.png')
            fig.savefig(fname, dpi=300, bbox_inches='tight')
            print ('saving --»' ,fname)
            plt.clf()
            plt.close()
            del early_late, early_data, late_data, elec, elec_label, freq_name, da, daperm, pvalue, label
            


-> Olfacto loaded
early shape:  (16, 29) late shape:  (16, 28)
elec  a2-a1 elec_label  Amg-pPirT
rand early matrix (16, 28)
Size of the concatenated data:  (57, 16) Number of features :  16
Size of label for classif:  57
decoding accuracy (10, 16) pvalues  (16,) daperm (100, 16)
71.9298245614
saving --» /media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/classified/8_Classif_Explo_Early_Late_700ms_200ms_across_time__LDA/LEFC_delta_Amg-pPirT_(0)_0.01.png
early shape:  (16, 29) late shape:  (16, 28)
elec  a2-a1 elec_label  Amg-pPirT
rand early matrix (16, 28)
Size of the concatenated data:  (57, 16) Number of features :  16
Size of label for classif:  57
decoding accuracy (10, 16) pvalues  (16,) daperm (100, 16)
70.1754385965
saving --» /media/karim/Datas4To/Analyses_Intra_EM_Odor/Olfacto/classified/8_Classif_Explo_Early_Late_700ms_200ms_across_time__LDA/LEFC_theta_Amg-pPirT_(0)_0.01.png
early shape:  (16, 29) late shape:  (16, 28)
elec  a2-a1 elec_label  Amg-pPirT
rand early matrix (16